<center>
<H1> Calling others languages in <H1>
<img src="julia.png" width="400">
</center> 
    
- source available at https://github.com/aitzkora/nahasketa/tree/master/julia/interfaces 
- disclaimer :
 - we address only the **Julia** calling other code part
 - only tested under Linux      
    

# Outline
## calling C code
## calling Fortran code 
## calling Python

# Calling  C code 

the basic julia syntax to interface with C code is 

```julia
ccall((:funcName, library), returnType, (argType1, argType2, ...), (argVal1, argVal2, ...))
```
## Remarks 
- library is a _formally_ a string :
 - you could use `"./mylib.so"`   
 - but ⚠ you **could not** use `string(pwd(),"/mylib.so")` ⚠ 
- to use a library which is not in `.`, add the path to `LD_LIBRARY_PATH` before launching **Julia**

# Basic example 
- we want to call a scalar function taking one int and returning one int
- we write the C function and compile it

In [30]:
io = open("C/skel.c","w")
write(io, "int ajoute2(int x) { return x+2; }")
close(io)
run(`gcc -o C/ajoute2.so --shared C/skel.c`);

- and we call it in Julia

In [31]:
w = ccall((:ajoute2, "ajoute2.so"), Int32, (Int32,), 12)
run(`rm C/ajoute2.so C/skel.c`)
println("w = $w")

w = 14


# Example with a matrix
- We can call the following C code
```C
#include <inttypes.h>
double mysum(int64_t m, int64_t n, double *x)
{
   double s = 0.;
   int i,j;
   for(i = 0; i < m; ++i)
       for(j = 0; j < n; ++j)
            s += x[i * n + j];
   return s;
}
```
- using 

```julia
a = reshape(collect(1.:10.), 5, 2)
s = ccall((:mysum, "./mysum.so"), Float64, (Int64, Int64, Ptr{Float64}), size(a, 1), size(a, 2), a)
```

## Notes : 

- we use `Ptr{Float64}` to pass a matrix to the function
- using standards types like `int64_t` is not mandatory, but enforces compatibility trough more portability 
- we could also use `Cdouble` and `Cint` in Julia code

# Calling a Julia Function in the C code called by Julia 
- using a function ptr `double (*fun) (double)` in C code

```C
#include <inttypes.h>

double apply_f(double (*fun)(double), int64_t x_n , double *x, double * y)
{
  int64_t i;
  for(i = 0; i < x_n; ++i)
      i[y] = (*fun)(i[x]);
}
```
and the `@cfuntion@` julia macro as in 

```julia
a = collect(1.:10.)
fun = @cfunction(x->x*x, Float64, (Float64,))
b = zeros(10)
s = ccall((:apply_f, "libjuliafun.so"), Cvoid, (Ptr{Cvoid}, Cint, Ptr{Cdouble}, Ptr{Cdouble}), fun, 10, a, b)
println("b = $b")
```
one could call julia function into the C code

In [4]:
include("C/test_apply_f.jl")

b = [1.0, 4.0, 9.0, 16.0, 25.0, 36.0, 49.0, 64.0, 81.0, 100.0]


# Basic C++ support
- we can use C++ code, as long as we provided a `extern "C"` interface for the entry function 

```C
#include <cinttypes>
#include <vector>
#include <numeric>

extern "C" {
double mysum2(int64_t n, double *x);
};

double mysum2(int64_t n, double *x)
{
    double s = 0;
    std::vector<double> t(x, x + n);
    for(auto & z : t) s+= z;
    return s;
}
```
can be called as 

```julia
a = collect(1.:10.)
s = ccall((:mysum2, "libmysum2.so"), Float64, (Int64, Ptr{Float64}), size(a, 1), a)
println("s = $s")
```

In [3]:
include("C/test_mysum2.jl")

s = 55.0
